In [10]:
import pd as pd
from matplotlib import pyplot as plt
import os
import glob
from zipfile import ZipFile
import torch
from torch.utils.data import Dataset, DataLoader
from torch import nn
from torch.utils.data.sampler import SubsetRandomSampler
import torchvision
import torchvision.datasets as dataset
import torchvision.transforms as transforms
from PIL import Image


In [9]:
IMAGE_SIZE= 224
exp_name= "sex_eyeglass"


/local/scratch/a/rahman89/PycharmProjects/modularCelebA/modularTraining/data_generation


In [19]:


# data_path = sorted(glob.glob('/local/scratch/a/rahman89/Datasets/CelebAMask-HQ/CelebAMask-HQ/CelebA-HQ-img/*.jpg')) # This was a bug. It did not sort the images as 0,1,2,...
data_path = '/local/scratch/a/rahman89/Datasets/CelebAMask-HQ/CelebAMask-HQ/CelebA-HQ-img/'
label_path = "/local/scratch/a/rahman89/Datasets/CelebAMask-HQ/CelebAMask-HQ/CelebAMask-HQ-attribute-anno.txt"
sep=" "
rowst=2
colst=2

label_list = open(label_path).readlines()[rowst:]


data_label = []
for i in range(len(label_list)):
    each_row= label_list[i].split(sep)
    data_label.append(each_row)



image_names=[]
for m in range(len(data_label)):
    image_names.append(data_label[m][0])  #image id
    data_label[m] = data_label[m][colst:]  # rest of the labels
    data_label[m] = [int(p) for p in data_label[m]]

attributes = open(label_path).readlines()[1].split(' ')
attributes[-1]= attributes[-1].strip('\n')




In [20]:
import numpy as np
import pandas as pd
labels= np.array(data_label)

data_dict={}
for iter, att in enumerate(attributes):
    data_dict[att] = labels[:,iter]

data_dict['image_id']= image_names
df = pd.DataFrame(data_dict)
df = df.replace([-1], 0)
cur_attrs=['Attractive', 'Bald',  'Eyeglasses',  'Male', 'Mustache', 'Pale_Skin',  'Wearing_Lipstick', 'Young']
all_labels= ['image_id']+ cur_attrs
df = df[all_labels]
df

,image_id,Attractive,Bald,Eyeglasses,Male,Mustache,Pale_Skin,Wearing_Lipstick,Young
0,0.jpg,1,0,0,0,0,0,1,1
1,1.jpg,1,0,0,0,0,0,1,1
2,2.jpg,1,0,0,0,0,0,1,1
3,3.jpg,1,0,0,0,0,0,1,1
4,4.jpg,0,0,0,0,0,0,0,1
...,...,...,...,...,...,...,...,...,...
29995,29995.jpg,0,0,1,1,1,0,0,0
29996,29996.jpg,0,0,0,1,0,0,0,1
29997,29997.jpg,0,0,0,0,0,0,0,1
29998,29998.jpg,1,0,0,0,0,0,1,1


# Generating dataset of nk samples


Train data distribution

	Eyeglass=0	Eyeglass=1
Male=0	3200	100
Male=1	1000	1080

	Eyeglass=0	Eyeglass=1
Male=0	0.60	0.018
Male=1	0.18	0.20

Total = 5380
P(Eye=1)=0.22
P(Male=0)= 0.61

In [35]:

m0e0= 3200
m0e1=100
m1e0=1000
m1e1= 1080

df_old_female= df.loc[(df['Male'] == 0) & (df['Eyeglasses']==0)][0:m0e0]
df_young_female= df.loc[(df['Male'] == 0) & (df['Eyeglasses']==1)][0:m0e1]
df_old_male= df.loc[(df['Male'] == 1) & (df['Eyeglasses']==0)][0:m1e0]
df_young_male= df.loc[(df['Male'] == 1) & (df['Eyeglasses']==1)][0:m1e1]



selected_df=  pd.concat([df_old_female,df_young_female, df_old_male, df_young_male])
train_df = selected_df.sample(frac=1).reset_index(drop=True)
train_df


,image_id,Attractive,Bald,Eyeglasses,Male,Mustache,Pale_Skin,Wearing_Lipstick,Young
0,283.jpg,0,0,0,1,0,0,0,1
1,2942.jpg,1,0,0,0,0,0,1,1
2,1542.jpg,1,0,0,0,0,0,1,1
3,2632.jpg,1,0,0,0,0,0,1,1
4,12803.jpg,0,0,1,1,0,0,0,0
...,...,...,...,...,...,...,...,...,...
5375,97.jpg,0,0,0,1,0,0,0,0
5376,1640.jpg,1,0,0,0,0,0,0,1
5377,948.jpg,0,0,0,1,0,0,0,1
5378,2150.jpg,0,0,0,0,0,0,1,1


In [36]:
len(df_old_female), len(df_young_female), len(df_old_male), len(df_young_male),

(3200, 100, 1000, 1080)

In [37]:
for lb in cur_attrs:
    print(lb, len(train_df[train_df[lb]==1]))

Attractive 2581
Bald 204
Eyeglasses 1180
Male 2080
Mustache 370
Pale_Skin 202
Wearing_Lipstick 2905
Young 3827


In [38]:
file_name= f'/local/scratch/a/rahman89/PycharmProjects/modularCelebA/{exp_name}/base_celeba_train.csv'
train_df.to_csv(file_name, encoding='utf-8', index=False)
print('saved at :',file_name)

saved at : /local/scratch/a/rahman89/PycharmProjects/modularCelebA/sex_eyeglass/base_celeba_train.csv


In [39]:
from tqdm import tqdm
import pickle


def save_labels(dom_name, true_data):
	save_folder= f'/local/scratch/a/rahman89/PycharmProjects/modularCelebA/{exp_name}'
	save_loc =f'{save_folder}/8_attribute_celeba_{dom_name}.pkl'
	print('Going to save at', save_loc)
	with open(save_loc, 'wb') as f:
		pickle.dump(true_data, f)



def save_images(dom_name, true_data):
	save_folder= f'/local/scratch/a/rahman89/PycharmProjects/modularCelebA/{exp_name}'
	save_loc =f'{save_folder}/images_celeba_{dom_name}.pkl'
	print('Going to save at', save_loc)
	with open(save_loc, 'wb') as f:
		pickle.dump(true_data, f)

# saving training data and validation data as pkl file here.
transform = transforms.Compose(
            [transforms.Resize(IMAGE_SIZE),
             transforms.ToTensor(),
             transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5))
             ]
        )


save_folder= f'/local/scratch/a/rahman89/PycharmProjects/modularCelebA/{exp_name}'
os.makedirs(save_folder, exist_ok=True)


label_path= f'/local/scratch/a/rahman89/PycharmProjects/modularCelebA/{exp_name}/base_celeba_train.csv'
sampled_labels = pd.read_csv(label_path)

true_dataset = {}
for lb in cur_attrs:
    true_dataset[lb]= torch.tensor(sampled_labels[lb].values)


save_labels('domain1', true_dataset)

img_folder= '/local/scratch/a/rahman89/Datasets/CelebAMask-HQ/CelebAMask-HQ/CelebA-HQ-img/'
images =[]
for id in tqdm(sampled_labels['image_id']):
    img_path = f'{img_folder}/{id}'
    cur_im = Image.open(img_path)
    image_tensor = transform(cur_im)
    images.append(image_tensor.unsqueeze(0))

true_dataset['I'] = torch.cat(images)

save_images(dom_name= "domain1", true_data= true_dataset)



Going to save at /local/scratch/a/rahman89/PycharmProjects/modularCelebA/sex_eyeglass/8_attribute_celeba_domain1.pkl


100%|██████████| 5380/5380 [00:35<00:00, 151.28it/s]


Going to save at /local/scratch/a/rahman89/PycharmProjects/modularCelebA/sex_eyeglass/images_celeba_domain1.pkl
